In [7]:
#Import Dependencies
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)



2.3.0


In [20]:
#read in the model csv
model_df = pd.read_csv("modelPython.csv")
model_df.columns
chosen = ['careerRec', 'careerRecYds', 'careerYPC','Conf']

In [29]:
#!/usr/bin/env python
# coding: utf-8

# Import Dependencies
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


def run_model(chosenstats):

    # read in the model csv
    model_df = pd.read_csv("modelPython.csv")

    # read in 2020 drafted players
    predict2020 = pd.read_csv('draft2020.csv')

    # Convert Age, weight, and height columns to floats
    model_df[['Wt', 'height']] = model_df[['Wt', 'height']].astype(float)
    predict2020[['Wt', 'height']] = predict2020[['Wt', 'height']].astype(float)

    # add required extras to the list of variable for the df
    train_stats = chosenstats.copy()
    train_stats.append('FPTS')

    # list of 2020 players that the model is predicting
    subjects2020 = predict2020[['ID','Player_x','FirstNm','LastNm']]

    # create df's with only the variables chosen
    train_df = model_df[train_stats]
    predict2020 =predict2020[chosenstats]

    #drop categorical columns that shouldn't be made to dummy variables

    # Create dummy variables for
    train_df = pd.get_dummies(train_df)
    predict2020 = pd.get_dummies(predict2020)
    
    print(train_df)
    print(predict2020)
    # drop 1 columns associated with each set of dummy variables created
    if 'Class' in chosenstats:
        print("found Class")
        train_df=train_df.drop(columns = ['Class_SR'])
        predict2020=predict2020.drop(columns = ['Class_SR'])
    else:
        print("No class chosen")

    if 'Conf' in chosenstats:
        print("found Conf")
        train_df = train_df.drop(columns = ['Conf_ACC'])
        predict2020=predict2020.drop(columns = ['Conf_ACC'])
    else:
        print("no Conf Chosen")

    
    #use only players that scored some points
    train_df = train_df[model_df['FPTS']!=0]
    
    

    # define x the inputs and y the actuals
    X = train_df.drop(columns = ['FPTS'])
    y = train_df['FPTS'].values.reshape(-1,1)
    print(X.shape, y.shape,predict2020.shape)

    #save the size of the predictors to pass into the model later
    size = len(X.columns)


    # Split the data into training and testing
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8,random_state=27)
    
    
    def build_model():
      model = keras.Sequential([
        layers.Dense(64, activation='relu', input_dim=size),
        layers.Dense(1)
      ])

      optimizer = tf.keras.optimizers.RMSprop(0.001)

      model.compile(loss='mse',
                    optimizer=optimizer,
                    metrics=['mae', 'mse'])
      return model
    
    # build model for all data
    model = build_model()
    
    model.fit(
    X_train,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=0
    )
    
    projectedpts = model.predict(predict2020)
    
    subjects2020['projected points']=projectedpts
    subjects2020.sort_values('projected points', ascending = False, inplace =True)
    subjects2020.drop(columns = ['FirstNm','LastNm','ID'], inplace = True)
    subjects2020.reset_index(inplace = True)
    subjects2020.drop(columns = ['index'],inplace = True)
    subjects2020.reset_index(inplace = True)

    subjects2020['Rank']=subjects2020['index']+1

    # reformat data for web display
    web_display = subjects2020[['Rank','Player_x','projected points']]

    # get data in a format to write to a file that can be read by webpage
    data = web_display.to_dict('dict')

    # with open('static/js/custmodel.js', 'w') as writer:
    #     writer.write("customdata=")
    # #     for line in data:
    # #         writer.write(line)
    #     json.dump(data, writer)
    # #     writer.write("]")
    return data







In [30]:
data = run_model(chosen)

     careerRec  careerRecYds  careerYPC   FPTS  Conf_ACC  Conf_American  \
0          161          1743       10.8   15.6         0              0   
1          101          1910       18.9  646.0         0              0   
2          132          1931       14.6   25.1         0              0   
3          138          1752       12.7   28.2         0              0   
4          259          3231       12.5  375.9         0              0   
..         ...           ...        ...    ...       ...            ...   
200        147          2114       14.4  235.0         0              0   
201        203          3085       15.2    0.0         0              0   
202        133          1956       14.7    0.0         0              0   
203        214          3177       14.8  237.6         0              0   
204        109          1672       15.3   22.7         0              0   

     Conf_Big 12  Conf_Big East  Conf_Big Ten  Conf_CUSA  Conf_Ind  Conf_MAC  \
0              0   

ValueError: in user code:

    C:\Users\walsh\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1462 predict_function  *
        return step_function(self, iterator)
    C:\Users\walsh\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\walsh\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\walsh\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\walsh\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\walsh\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1445 run_step 